In [ ]:
%pip install --upgrade boto3

In [1]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime

In [2]:
my_region = boto3.session.Session().region_name # set the region of the instance
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the eu-west-1 region. You will use the 685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [3]:
bucket_name = 'ykarle-s3-test-bucket1' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')

try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': my_region})
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [4]:
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

Success: downloaded bank_clean.csv.


In [5]:
try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.


In [6]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41188 entries, 0 to 41187
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   age                            41188 non-null  int64
 1   campaign                       41188 non-null  int64
 2   pdays                          41188 non-null  int64
 3   previous                       41188 non-null  int64
 4   no_previous_contact            41188 non-null  int64
 5   not_working                    41188 non-null  int64
 6   job_admin.                     41188 non-null  int64
 7   job_blue-collar                41188 non-null  int64
 8   job_entrepreneur               41188 non-null  int64
 9   job_housemaid                  41188 non-null  int64
 10  job_management                 41188 non-null  int64
 11  job_retired                    41188 non-null  int64
 12  job_self-employed              41188 non-null  int64
 13  job_services    

In [7]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), 
                                 [int(0.7 * len(model_data))])

print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [11]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)

prefix = 'sagemaker/DEMO-xgboost-dm'
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), 
                                                content_type='csv')

In [12]:
role = get_execution_role()
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(xgboost_container, 
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket_name, prefix),
                                    sagemaker_session=sess)
    
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

In [13]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-01-14-10-15-26-842


2023-01-14 10:15:27 Starting - Starting the training job...
2023-01-14 10:15:52 Starting - Preparing the instances for training......
2023-01-14 10:16:56 Downloading - Downloading input data...
2023-01-14 10:17:26 Training - Downloading the training image...
2023-01-14 10:17:56 Training - Training image download completed. Training in progress....
2023-01-14 10:18:27 Uploading - Uploading generated training modelArguments: train
[2023-01-14:10:18:13:INFO] Running standalone xgboost training.
[2023-01-14:10:18:13:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-01-14:10:18:13:INFO] File size need to be processed in the node: 3.38mb. Available memory size in the node: 8603.98mb
[2023-01-14:10:18:13:INFO] Determined delimiter of CSV input is ','
[10:18:13] S3DistributionType set as FullyReplicated
[10:18:13] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[10:18:13] src/tree/updater_prune.cc:74: tree pruning end

In [14]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2023-01-14-10-20-11-072
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-01-14-10-20-11-072
INFO:sagemaker:Creating endpoint with name xgboost-2023-01-14-10-20-11-072


------!

In [15]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [16]:
cm = pd.crosstab(index=test_data['y_yes'], 
                 columns=np.round(predictions_array), 
                 rownames=['Observed'], 
                 colnames=['Predicted'])
tn = cm.iloc[0,0]
fn = cm.iloc[1,0]
tp = cm.iloc[1,1]
fp = cm.iloc[0,1]
p = (tp+tn)/(tp+tn+fp+fn)*100

print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10769)    37% (167)
Purchase        10% (1133)     63% (288) 



In [17]:
xgb_predictor.delete_endpoint(delete_endpoint_config=True)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

INFO:sagemaker:Deleting endpoint configuration with name: xgboost-2023-01-14-10-20-11-072
INFO:sagemaker:Deleting endpoint with name: xgboost-2023-01-14-10-20-11-072


[{'ResponseMetadata': {'RequestId': 'JV3FF0EJTM3MRMKD',
   'HostId': 'dKArXQDxpvUWOzznHhKmB1OP0I529pt6+otwTORuBWGX+1Q7G4MR/f66slFCjpS91bwFTwPiSrA=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'dKArXQDxpvUWOzznHhKmB1OP0I529pt6+otwTORuBWGX+1Q7G4MR/f66slFCjpS91bwFTwPiSrA=',
    'x-amz-request-id': 'JV3FF0EJTM3MRMKD',
    'date': 'Sat, 14 Jan 2023 10:25:21 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2023-01-14-10-15-26-842/debug-output/training_job_end.ts'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2023-01-14-10-15-26-842/profiler-output/system/incremental/2023011410/1673691360.algo-1.json'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2023-01-14-10-15-26-842/profiler-output/framework/training_job_end.ts'},
   {'Key': 'sagemaker/DEMO-xgboost-dm/output/xgboost-2023-01-14-10-15-2